<a href="https://colab.research.google.com/github/21Alul21/HouzStackApi_PortfollioProject/blob/main/uk_model_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
# importing necessary libraries|
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [230]:
!pip install xgboost==2.1.0

In [231]:
# loadinding the dataset

df = pd.read_csv('./datasets/uk_housing_rentals.csv')

In [232]:
df

,Unnamed: 0,Description,Location,Number of Rooms,Price
0,0,1 Bedroom Flat For Rent,"Birmingham, West Midla...",1 bedroom,£ 625 ...
1,1,1 Bedroom Flat For Rent,"Birmingham, West Midla...",1 bedroom,£ 625 ...
2,2,"4B Paddington Basin, London, W2","London, Greater London...",3 bedrooms,"£ 6,000 ..."
3,3,2 Bedroom Terraced House to Rent in Albert Roa...,"London, Greater London...",2 bedrooms,"£ 1,400 ..."
4,4,2 Bedroom Terraced House to Rent in Albert Roa...,"London, Greater London...",2 bedrooms,"£ 1,400 ..."
...,...,...,...,...,...
39034,2602,apartment for rent in London,"London, Greater London...",2 bedrooms,"£ 2,297 ..."
39035,2603,Apartment for rent in London,"London, Greater London...",1 bedroom,£ 462 ...
39036,2604,Apartment for rent in London,"London, Greater London...",1 bedroom,£ 462 ...
39037,2605,"Apartment in London, Greater London","London, Greater London...",2 bedrooms,"£ 26,678 ..."


## Exploratory Data Analysis on the Dataset

In [233]:
# short description of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39039 entries, 0 to 39038
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       39039 non-null  int64 
 1   Description      39039 non-null  object
 2   Location         39039 non-null  object
 3   Number of Rooms  39039 non-null  object
 4   Price            39039 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [234]:
# statistical description of dataset
df.describe()

,Unnamed: 0
count,39039.000000
mean,14056.481826
std,10376.635315
min,0.000000
25%,3654.500000
50%,13414.000000
75%,23173.500000
max,32933.000000


In [235]:
# checking for missing values in dataset
df.isna().sum()

Unnamed: 0         0
Description        0
Location           0
Number of Rooms    0
Price              0
dtype: int64

In [236]:
# checking for duplicate rows in dataset

df.duplicated().sum()

1188

In [237]:
df.drop_duplicates(inplace=True)

In [238]:
df.duplicated().sum()

0

In [239]:
df.columns.unique()

Index(['Unnamed: 0', 'Description', 'Location', 'Number of Rooms', 'Price'], dtype='object')

In [240]:
df.Price

0                                    £ 625             ...
1                                    £ 625             ...
2                                    £ 6,000           ...
3                                    £ 1,400           ...
4                                    £ 1,400           ...
                               ...                        
39034                                £ 2,297           ...
39035                                £ 462             ...
39036                                £ 462             ...
39037                                £ 26,678          ...
39038                                £ 1,900           ...
Name: Price, Length: 37851, dtype: object

In [241]:
df.shape

(37851, 5)

In [242]:
df.Location.unique()

array(['                        Birmingham, West Midlands                    ',
       '                        London, Greater London                    ',
       '                        Liverpool, Merseyside                    ',
       '                        Leeds, Leeds                    ',
       '                        Harrow, Harrow                    ',
       '                        Bromley, Bromley                    '],
      dtype=object)

In [243]:
df.drop(['Unnamed: 0', 'Description', 'Location'], axis=1,inplace=True)

In [244]:
df.shape

(37851, 2)

In [245]:
df.head()

,Number of Rooms,Price
0,1 bedroom,£ 625 ...
1,1 bedroom,£ 625 ...
2,3 bedrooms,"£ 6,000 ..."
3,2 bedrooms,"£ 1,400 ..."
4,2 bedrooms,"£ 1,400 ..."


In [246]:
# stripping the text from the cell values and converting to integer data type

df['Number of Rooms'] = df['Number of Rooms'].apply(lambda x: x.split(' ')[0])

def clean_price(price):
  value = ''
  for char in price:
    if char.isdigit():
      value += char
  return int(value)

df['Price'] = df['Price'].apply(clean_price)

In [247]:
df

,Number of Rooms,Price
0,1,625
1,1,625
2,3,6000
3,2,1400
4,2,1400
...,...,...
39034,2,2297
39035,1,462
39036,1,462
39037,2,26678


In [248]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37851 entries, 0 to 39038
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Number of Rooms  37851 non-null  object
 1   Price            37851 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 887.1+ KB


In [249]:
df['Number of Rooms'].unique()

array(['1', '3', '2', '4', '6', '5', 'Not', '34', '10', '27', '24', '7'],
      dtype=object)

In [250]:
# removing rows with value 'Not'

df = df[df['Number of Rooms'] != 'Not']


In [251]:
df['Number of Rooms'].unique()

array(['1', '3', '2', '4', '6', '5', '34', '10', '27', '24', '7'],
      dtype=object)

In [252]:
# converting 'NUmber of Rooms' column to integer
df['Number of Rooms'] = df['Number of Rooms'].astype(int)

/tmp/ipykernel_13405/3388982608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Number of Rooms'] = df['Number of Rooms'].astype(int)


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37767 entries, 0 to 39038
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Number of Rooms  37767 non-null  int64
 1   Price            37767 non-null  int64
dtypes: int64(2)
memory usage: 885.2 KB


## Model Training

In [254]:
X = df['Number of Rooms']
Y = df['Price']

In [255]:
# splitting the model for training and testing
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)

In [256]:
model = xgb.XGBRegressor()

In [257]:
model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [258]:
prediction = model.predict(x_test)

In [259]:
prediction

array([2985.4888, 3341.4053, 2985.4888, ..., 2985.4888, 2985.4888,
       1617.62  ], dtype=float32)

In [260]:
df_t = pd.DataFrame({'Number of Rooms': [2]})

In [261]:
model.predict(df_t)

array([3341.4053], dtype=float32)

In [262]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(prediction, y_test)


1310.1139475464158

In [266]:
# saving the model using pickle
import pickle

with open('model_uk.pkl', 'wb') as m:
    pickle.dump(model, m)

In [226]:
#import joblib
#joblib.dump('model', 'xgb_model_uk.plk')